# Code for: temporally aggregated model

In [16]:
using Plots, Measures

include("../src/LoadData.jl")
include("../src/PMMH.jl")
include("../src/MarginalPosterior.jl")
include("../src/Support.jl")

# Load data
Yin = loadData("NZCOVID_1APR2024");

# Calculate aggregated cases
Yin.WeeklyCases = repeat([0], length(Yin.Ct))
for ii = 1:length(Yin.Ct)
    if ii % 7 == 0
        Yin.WeeklyCases[ii] = sum(Yin.Ct[ii-6:ii])
    end
end

# Use only the first 100 days when fitting the model
Y = Yin[1:100,:];

In [17]:
function temporallyAggregatedModel(θ, Y::DataFrame, opts::Dict)
    
    # Extract frequently used options
    T = opts["T"]
    N = opts["N"]
    L = opts["L"]
    ω = opts["ω"]
    delayDist = opts["delayDist"]
    h = opts["forecastingHorizon"]
    
    # Initialise output matrices
    R = zeros(N, T+h)
    I = zeros(N, T+h)
    W = ones(N, T)
    
    # And predictive values
    μ = zeros(N, T+h) # Store expected cases to avoid resampling issues
    C = zeros(N, T+h)
    
    # Sample from initial distributions
    R[:,1] = rand(opts["pR0"], N)
    I[:,1] = rand(opts["pI0"], N)
    
    # Run the filter
    for tt = 2:T
        
        # Project according to the state-space model
        R[:,tt] = exp.(rand.(Normal.(log.(R[:,tt-1]), θ[1])))
        Λ = sum(I[:, (tt-1):-1:1] .* ω[1:(tt-1)]', dims=2) ./ sum(ω[1:(tt-1)])
        I[:,tt] = rand.(Poisson.(R[:,tt] .* Λ))
        
        # Weight according to the observation model, but only on the day that we observe data
        if tt % 7 == 0
            
            # Fetch expected reported cases
            μt = zeros(N)
            for ss = max(tt-6, 2):tt
                μt = μt .+ (sum(I[:,(ss-1):-1:1] .* delayDist[1:(ss-1)]', dims=2) ./ sum(delayDist[1:(ss-1)]))
            end
            
            # Calculate weights
            r = 1/θ[2]
            p = 1 ./ (1 .+ θ[2] * μt)
            W[:,tt] = pdf.(NegativeBinomial.(r, p), Y.WeeklyCases[tt])
            
            # Resample
            inds = wsample(1:N, W[:,tt], N; replace=true)
            R[:, max(tt - L, 1):tt] = R[inds, max(tt - L, 1):tt]
            I[:, max(tt - L, 1):tt] = I[inds, max(tt - L, 1):tt]

            # Store expected cases if we are finding predictive posterior
            if opts["predictiveValues"]
                μ[:,tt] = μt
                μ[:,max(tt-L,1):tt] = μ[inds,max(tt-L,1):tt]
            end
            
        end
        
    end
    
    # Fetch predictive values
    if opts["predictiveValues"]
        for tt = 2:T
            if tt % 7 == 0
                r = 1/θ[2]
                p = 1 ./ (1 .+ θ[2] * μ[:,tt])
                C[:,tt] = rand.(NegativeBinomial.(r, p))
            end
        end
    end
    
    # Fetch forecasting values
    if h > 0
        for ss = (T+1):(T+h)
            R[:,ss] = exp.(rand.(Normal.(log.(R[:,ss-1]), θ[1])))
            Λ = sum(I[:, ss:-1:1] .* ω[1:ss]', dims=2) ./ sum(ω[1:ss]) # normalise to avoid need for wind-in
            I[:,ss] = rand.(Truncated.(Poisson.(R[:,ss] .* Λ), 0, 1e6))
            if ss % 7 == 0
                μt = zeros(N)
                for uu = max(ss-6, 2):ss
                    μt = μt .+ (sum(I[:,(uu-1):-1:1] .* delayDist[1:(uu-1)]', dims=2) ./ sum(delayDist[1:(uu-1)]))
                end
                r = 1/θ[2]
                p = 1 ./ (1 .+ θ[2] * μt)
                C[:,ss] = rand.(NegativeBinomial.(r, p))
            end
        end
    end
    
    # Store output as three-dimensional array
    X = zeros(N, T+h, 3)
    X[:,:,1] = R
    X[:,:,2] = I
    X[:,:,3] = C
    
    # Forecast
    return(X, W)
    
end


temporallyAggregatedModel (generic function with 1 method)

In [18]:
opts = Dict(

    # Bootstrap filter options
    "T" => size(Y, 1), # Number of time-steps
    "N" => 1000, # Number of particles
    "L" => 50, # Fixed-lag resampling length
    "ω" => pdf.(Gamma(2.36, 2.74), 1:128), # Serial interval
    "delayDist" => pdf.(Gamma(5.72, 0.96), 1:200), # Observation delay distribution
    "pR0" => Uniform(0, 10), # Prior on Rt at t = 0
    "pI0" => DiscreteUniform(200, 600), # Prior on I at t = 0
    "predictiveValues" => false, # Whether to calculate predictive cases
    "forecastingHorizon" => 0, # Number of days to forecast

    # PMMH options
    "nChains" => 3, # Number of chains
    "chunkSize" => 100, # Number of iterations
    "maxChunks" => 50, # Maximum number of chunks
    "maxRhat" => 1.05,  # Stopping criterion: maximum Rhat value
    "minESS" => 100, # Stopping criterion: minimum effective sample size
    "showChunkProgress" => true, # Whether to show progress of each chunk
    "propStdDevInit" => sqrt.([0.01, 0.001]), # Initial proposal standard deviation (this is adaptively fit)
    "paramPriors" => [Uniform(0, 1), Uniform(0, 0.1)],
    "initialParamSamplers" => [Uniform(0.05, 0.2), Uniform(0.01, 0.02)],
    "paramLimits" => [(0, 1), (0, 0.1)],
    "paramNames" => ["σ", "ϕ"],

    # Marginal posterior options
    "posteriorNumberOfParticles" => 10000,
    "posteriorParamSamples" => 100

);

In [19]:
# Run PMMH (algorithm 2)
(θ, diag) = PMMH(temporallyAggregatedModel, Y, opts; verbose=false)
chains = Chains(θ, opts["paramNames"])

Chains MCMC chain (1200×2×3 Array{Float64, 3}):

Iterations        = 1:1:1200
Number of chains  = 3
Samples per chain = 1200
parameters        = σ, ϕ

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64 

           σ    0.0704    0.0237     0.0004    0.0016   160.7627    1.0077
           ϕ    0.0084    0.0099     0.0002    0.0009   101.4389    1.0080

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

           σ    0.0393    0.0548    0.0669    0.0793    0.1318
           ϕ    0.0012    0.0033    0.0052    0.0093    0.0384


In [20]:
# Fetch marginal posterior
opts["predictiveValues"] = true
opts["forecastingHorizon"] = 28
X = marginalPosterior(temporallyAggregatedModel, θ, Y, opts);

In [21]:
# Save model results
using HDF5
if isfile("temp/temporallyAggregatedResults.h5")
    rm("temp/temporallyAggregatedResults.h5")
end
h5write("temp/temporallyAggregatedResults.h5", "X", X)
h5write("temp/temporallyAggregatedResults.h5", "theta", θ)